## 組合せ最適化問題
量子コンピュータで最適化問題を解くには、イジングモデルといわれる物理モデルを利用する。

## QUBO定式化
QUBOは問題の答えが小さいほうが正解になるように設定された式です。式の形は、

$$
QUBO = -\sum_i h_i q_i -\sum_{i,j}J_{ij}q_iq_j
$$

となっている。iとjは点を表し、hはバイアス（局所磁場）、Jは相互作用と呼ばれます。この式ではqは量子ビットを表し0か1を取ります（イジングの場合は+1か-1）。
私たちはhとJを問題として設定し、qの値を求めます。

## 問題設定の仕方
問題の設定の仕方は、グラフ問題というものに問題を落とすことで計算できますが、いくつか問題を解くことでコツをつかめます。

主に問題のコスト関数は二種類の式を考える必要があります。

１．小さくしたいコスト関数  
２．満たすべき条件（制約条件）

この二つを別々に設計し、つなげることで実装できます。片方しかない式もあります。

## 使うツール

networkx（ネットワークグラフを書く）  
matplotlib（各種のグラフを書く）  
numpy（数値ライブラリ）  

In [1]:
!pip install --quiet networkx matplotlib

## 2-1.二値整数計画問題
こちらはある連立方程式を満たしたうえである計算を最小化する、制約付きの最小値問題です。  
より正確には、$Sx = b$を満たすベクトル$x$のうち、$cx$を最小にするxを探します。

### QUBO
この問題は二つの式から成り立ちます。連立方程式の制約を満たすための$H_A$と最小コストを求めるための$H_B$です。

連立方程式を満たす式
$$
H_A = \sum_{j=1}^m[b_j - \sum_{i=1}^NS_{ji}x_i]^2
$$

ベクトルの計算を最小化する式
$$
H_B = - \sum_{i=1}^Nc_ix_i
$$

つないだ式。つなぐときには調整パラメーターが必要になりますので、$\lambda$を付けました。
$$
H = H_A + \lambda*H_B = \sum_{j=1}^m[b_j - \sum_{i=1}^NS_{ji}x_i]^2 - \lambda*\sum_{i=1}^Nc_ix_i
$$

### 問題設定

下記を満たすxのうち、

$$
\begin{bmatrix}
3 & 2 & 1 \\
5 & 2 & 3
\end{bmatrix}
\begin{bmatrix}
x_0\\x_1\\x_2
\end{bmatrix}
=
\begin{bmatrix}
3\\5
\end{bmatrix}
$$

次の式を最小にするようなxを求める。

$$
\begin{bmatrix}
1&2&1
\end{bmatrix}
\begin{bmatrix}
x_0\\x_1\\x_2
\end{bmatrix}
$$

制約条件を満たすとき、QUBO式は0になります。そのため、0にならなかったら制約を満たしていないというのがすぐにわかります。制約を満たしてなかったらそもそも答えとして成立しません。

In [3]:
from tytan import *
import sympy as sym

# 変数を定義 q_0, q_1, q_2
q = sym.symbols("q_{0:3}")

#式を別々に用意します。
HA = (3*q[0]+2*q[1]+q[2]-3)**2 + (5*q[0]+2*q[1]+3*q[2]-5)**2
HB = -(q[0]+2*q[1]+q[2])

#つけるときに今回はConstraint関数を使うことであとで簡単に確かめができます。Constraintは制約条件の方だけにつけます。
M = 10
H = HA + M*HB

# Compileクラスを使用して、QUBOを取得
Q, offset = qubo.Compile(H).get_qubo()

print(Q,offset)

# サンプラーを選択
solver = sampler.SASampler()

#クラウドサンプラーの場合
#API_KEY = "API key"
#solver = sampler.NQSSampler()
#result = solver.run(Q, api_key=API_KEY)

# 計算
result = solver.run(Q, shots=100)
print(result)

{('q_{0}', 'q_{0}'): -44, ('q_{1}', 'q_{1}'): -44, ('q_{2}', 'q_{2}'): -36, ('q_{1}', 'q_{2}'): 16, ('q_{0}', 'q_{1}'): 32, ('q_{0}', 'q_{2}'): 36} 34
[[{'q_{0}': 0.0, 'q_{1}': 1.0, 'q_{2}': 1.0}, -64.0, 42], [{'q_{0}': 1.0, 'q_{1}': 1.0, 'q_{2}': 0.0}, -56.0, 58]]


In [4]:
print("Sample =", result[0][0])
print("Cost =", result[0][1] + offset)

Sample = {'q_{0}': 0.0, 'q_{1}': 1.0, 'q_{2}': 1.0}
Cost = -30.0000000000000


011が正解で、コストが-30.0となりました。

## 2-2. 巡回セールスマン問題
巡回セールスマン問題はとても有名な問題です。解いてみましょう。巡回セールスマン問題は都市を一筆書きで順番に回り、一番回った距離が短かったのを正解とする組合せ最適化問題です。二回同じ都市を回ってはいけません。

### QUBO
こちらは回る距離が小さくなるコスト関数１つと制約条件が２種類あります。

$$
H = \sum_{v=1}^n\left( 1-\sum_{j=1}^N x_{v,j} \right)^2 + \sum_{j=1}^N\left(1-\sum_{v=1}^Nx_{v,j} \right)^2 + B\sum_{(u,v)\in E}W_{u,v}\sum_{j=1}^N x_{u,j} x_{v,j+1}
$$

### 問題設定
実装してみます。ABCD四つの都市があり、それぞれの都市の間の距離を設定した問題を実装します。これらの都市を二回回らず一回で最短距離を求めます。
![img](./img/tsp1.png)

Nは都市数、Wは都市間の距離です。uとvは都市の番号。jは回る順番です。式だけだとわかりづらいので前の二つの制約条件を図で見てみます。利用する量子ビットは都市数に対して$N^2$となります。一つの量子ビットに回る都市とその順番を割り当てます。

下の図ではAからDまでの都市が行に、回る順番が列に指定されています。全部で4x4=16量子ビットあります。0のものは使わず、1のものをつかい、1となっている量子ビットの都市と回る順番が採用されます。

一つ目の制約条件は一つの都市は一度だけ回るです。なので順番が複数あってはいけません。なので各列で1つだけ1、残りは0になるはずです。これをすべての列で行います。

![img2](./img/tsp2.png)

二つ目の条件は一つの順番は一度だけ出現するです。上記の行の条件だけではすべて同じ列に集まってしまった場合も成立してしまいます。なので、次に同じように列の条件を加えます。

![tsp3](./img/tsp3.png)

この二つで制約条件は終わりです。最後に距離を含んだコスト関数を加えて終わりです。

$$
H = \sum_{v=1}^n\left( 1-\sum_{j=1}^N x_{v,j} \right)^2 + \sum_{j=1}^N\left(1-\sum_{v=1}^Nx_{v,j} \right)^2 + B\sum_{(u,v)\in E}W_{u,v}\sum_{j=1}^N x_{u,j} x_{v,j+1}
$$


制約条件とコスト関数はバランスを調整する必要があり、調整変数Bが加えられてます。これは自分で探索する必要があります。量子ビットを左上から右下へ通し番号を0から15まで付けます。一つ目の制約条件は、

In [7]:
from tytan import *
import sympy as sym
import numpy as np

# 変数を定義
q = sym.symbols("q_{0:16}")


#行の制約
H1 = (1-q[0]-q[1]-q[2]-q[3])**2 + (1-q[4]-q[5]-q[6]-q[7])**2 + (1-q[8]-q[9]-q[10]-q[11])**2 + (1-q[12]-q[13]-q[14]-q[15])**2

#列の制約
H2 = (1-q[0]-q[4]-q[8]-q[12])**2 + (1-q[1]-q[5]-q[9]-q[13])**2 + (1-q[2]-q[6]-q[10]-q[14])**2 + (1-q[3]-q[7]-q[11]-q[15])**2

次に全体の距離を設定します。距離は問題からとります。距離の計算はちょっとめんどいかもしれません。

In [8]:
#ABCDの順
d = np.ones((16,16))*99

#A=>B
d[0][5] = d[1][6] = d[2][7] = d[3][4] = 2
d[5][0] = d[6][1] = d[7][2] = d[4][3] = 2

#A=>C
d[0][9] = d[1][10] = d[2][11] = d[3][8] = 1
d[9][0] = d[10][1] = d[11][2] = d[8][3] = 1

#A=>D
d[0][13] = d[1][14] = d[2][15] = d[3][12] = 3
d[13][0] = d[14][1] = d[15][2] = d[12][3] = 3

#B=>A
d[4][1] = d[5][2] = d[6][3] = d[7][0] = 2
d[1][4] = d[2][5] = d[3][6] = d[0][7] = 2

#B=>C
d[4][9] = d[5][10] = d[6][11] = d[7][8] = 4
d[9][4] = d[10][5] = d[11][6] = d[8][7] = 4

#B=>D
d[4][13] = d[5][14] = d[6][15] = d[7][12] = 2
d[13][4] = d[14][5] = d[15][6] = d[12][7] = 2

#C=>A
d[8][1] = d[9][2] = d[10][3] = d[11][0] = 1
d[1][8] = d[2][9] = d[3][10] = d[0][11] = 1

#C=>B
d[8][5] = d[9][6] = d[10][7] = d[11][4] = 4
d[5][8] = d[6][9] = d[7][10] = d[4][11] = 4

#C=>D
d[8][13] = d[9][14] = d[10][15] = d[11][12] = 2
d[13][8] = d[14][9] = d[15][10] = d[12][11] = 2

#D=>A
d[12][1] = d[13][2] = d[14][3] = d[15][0] = 3
d[1][12] = d[2][13] = d[3][14] = d[0][15] = 3

#D=>B
d[12][5] = d[13][6] = d[14][7] = d[15][4] = 2
d[5][12] = d[6][13] = d[7][14] = d[4][15] = 2

#D=>C
d[12][9] = d[13][10] = d[14][11] = d[15][8] = 2
d[9][12] = d[10][13] = d[11][14] = d[8][15] = 2

In [9]:
Hd = 0
for i in range(16):
    for j in range(16):
        Hd += d[i][j]*q[i]*q[j]

In [14]:
M=0.001

#constraintを二つ、コストを一つ
H = H1 + H2 + M*Hd

# Compileクラスを使用して、QUBOを取得
Q, offset = qubo.Compile(H).get_qubo()

#print(Q)

# サンプラーを選択
solver = sampler.SASampler()

#クラウドサンプラーの場合
#API_KEY = "API key"
#solver = sampler.NQSSampler()
#result = solver.run(Q, api_key=API_KEY)

# 計算
result = solver.run(Q, shots=100)
#print(result)

In [19]:
print("Sample =", result[0][0])


Sample = {'q_{0}': 0.0, 'q_{10}': 0.0, 'q_{11}': 1.0, 'q_{12}': 1.0, 'q_{13}': 0.0, 'q_{14}': 0.0, 'q_{15}': 0.0, 'q_{1}': 0.0, 'q_{2}': 1.0, 'q_{3}': 0.0, 'q_{4}': 0.0, 'q_{5}': 1.0, 'q_{6}': 0.0, 'q_{7}': 0.0, 'q_{8}': 0.0, 'q_{9}': 0.0}


In [20]:
print("Cost =", result[0][1])

Cost = -7.193999999999999


D=>B=>A=>Cの順番で解けました。